# Imports_and_Preparation

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import ast

python(18611) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [3]:
print("CUSTOMERS:")
print(customers_df.head(), "\n")
print("PRODUCTS:")
print(products_df.head(), "\n")
print("TRANSACTIONS:")
print(transactions_df.head(), "\n")

CUSTOMERS:
  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15 

PRODUCTS:
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31 

TRANSACTIONS:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  20

In [4]:
print("Shape of Customers:", customers_df.shape)
print("Shape of Products :", products_df.shape)
print("Shape of Transactions:", transactions_df.shape)

Shape of Customers: (200, 4)
Shape of Products : (100, 4)
Shape of Transactions: (1000, 7)


In [5]:
print("\nMissing Values in Customers:\n", customers_df.isnull().sum())
print("\nMissing Values in Products:\n", products_df.isnull().sum())
print("\nMissing Values in Transactions:\n", transactions_df.isnull().sum())


Missing Values in Customers:
 CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64

Missing Values in Products:
 ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64

Missing Values in Transactions:
 TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [6]:
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'], errors='coerce')
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'], errors='coerce')

In [7]:
print("\nDuplicate Rows in Customers:", customers_df.duplicated().sum())
print("Duplicate Rows in Products:", products_df.duplicated().sum())
print("Duplicate Rows in Transactions:", transactions_df.duplicated().sum())


Duplicate Rows in Customers: 0
Duplicate Rows in Products: 0
Duplicate Rows in Transactions: 0


In [8]:
invalid_cust_ids = set(transactions_df['CustomerID']) - set(customers_df['CustomerID'])
print(f"\nInvalid CustomerIDs in Transactions (not in Customers): {invalid_cust_ids}")


Invalid CustomerIDs in Transactions (not in Customers): set()


In [9]:
invalid_prod_ids = set(transactions_df['ProductID']) - set(products_df['ProductID'])
print(f"Invalid ProductIDs in Transactions (not in Products): {invalid_prod_ids}")

Invalid ProductIDs in Transactions (not in Products): set()


In [10]:
trans_cust_df = pd.merge(
    transactions_df, 
    customers_df[['CustomerID', 'Region']], 
    on='CustomerID', 
    how='left'
)

In [11]:
trans_cust_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,Region
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Europe
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Asia
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Europe
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,South America
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Europe
...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,South America
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,North America
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,North America
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Asia


In [12]:
full_df = pd.merge(
    trans_cust_df,
    products_df[['ProductID', 'Category']],
    on='ProductID',
    how='left'
)

In [13]:
full_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,Region,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Europe,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Asia,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Europe,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,South America,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Europe,Electronics
...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,South America,Electronics
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,North America,Electronics
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,North America,Electronics
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Asia,Electronics


In [14]:
print("\nMerged DataFrame (full_df) preview:")
print(full_df.head())


Merged DataFrame (full_df) preview:
  TransactionID CustomerID ProductID     TransactionDate  Quantity  \
0        T00001      C0199      P067 2024-08-25 12:38:23         1   
1        T00112      C0146      P067 2024-05-27 22:23:54         1   
2        T00166      C0127      P067 2024-04-25 07:38:55         1   
3        T00272      C0087      P067 2024-03-26 22:55:37         2   
4        T00363      C0070      P067 2024-03-21 15:10:10         3   

   TotalValue   Price         Region     Category  
0      300.68  300.68         Europe  Electronics  
1      300.68  300.68           Asia  Electronics  
2      300.68  300.68         Europe  Electronics  
3      601.36  300.68  South America  Electronics  
4      902.04  300.68         Europe  Electronics  


# FEATURE_ENGINEERING_FOR_LOOKALIKE_MODEL

In [15]:
customer_agg = transactions_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'TransactionID': 'count'
}).rename(columns={
    'TotalValue': 'total_spend',
    'Quantity': 'total_qty',
    'TransactionID': 'trans_count'
}).reset_index()

In [16]:
customer_agg['avg_order_value'] = customer_agg['total_spend'] / customer_agg['trans_count']

#Category_Spend_Distribution

In [17]:
tx_with_cat = pd.merge(
    transactions_df, 
    products_df[['ProductID', 'Category']], 
    on='ProductID', 
    how='left'
)

In [18]:
cat_spend = tx_with_cat.groupby(['CustomerID', 'Category'])['TotalValue'].sum().unstack(fill_value=0)

In [19]:
cat_spend_fraction = cat_spend.div(cat_spend.sum(axis=1), axis=0).fillna(0)

In [20]:
cust_profile = customers_df[['CustomerID', 'Region']].copy()

#Merge_all_Aggregated_data

In [21]:
cust_features = pd.merge(cust_profile, customer_agg, on='CustomerID', how='left')
cust_features = pd.merge(cust_features, cat_spend_fraction, on='CustomerID', how='left')

In [22]:
cust_features.fillna(0, inplace=True)

#Encode_Categorical_Columns

In [24]:
cust_features = pd.get_dummies(cust_features, columns=['Region'], prefix='Region')

# BUILD_FEATURE_MATRIX_AND_COMPUTE_SIMILARITIES

In [25]:
feature_cols = [col for col in cust_features.columns if col != 'CustomerID']

In [26]:
X = cust_features[feature_cols].values

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Compute_pairwise_cosine_similarity_among_all_customers

In [28]:
similarity_matrix = cosine_similarity(X_scaled)

In [29]:
index_to_customer = dict(enumerate(cust_features['CustomerID'].values))
customer_to_index = {v: k for k, v in index_to_customer.items()}

# FIND_TOP_3_LOOKALIKES_FOR_EACH_CUSTOMER

In [30]:
def get_top_k_similar(customer_id, k=3):
    """
    Given a customer_id, returns the top k most similar customer IDs and their similarity scores.
    Excludes the customer themselves.
    """
    cust_idx = customer_to_index[customer_id]
    
    sims = similarity_matrix[cust_idx]
    
    similar_indices = np.argsort(sims)[::-1]  
    
    similar_indices = similar_indices[similar_indices != cust_idx]
    
    top_k_indices = similar_indices[:k]
    
    result = []
    for idx in top_k_indices:
        sim_score = sims[idx]
        result.append((index_to_customer[idx], round(float(sim_score), 4)))
        
    return result

In [31]:
lookalike_dict = {}

In [32]:
desired_customers = [f"C{str(i).zfill(4)}" for i in range(1, 21)]

In [33]:
for cust_id in desired_customers:
    if cust_id in customer_to_index: 
        top_3 = get_top_k_similar(cust_id, k=3)
        lookalike_dict[cust_id] = top_3
    else:
        lookalike_dict[cust_id] = []

# SAVE_RESULTS_TO "Lookalike.csv"

In [34]:
data_for_csv = []
for cust_id, lookalikes in lookalike_dict.items():
    data_for_csv.append({
        'CustomerID': cust_id,
        'Lookalikes': str(lookalikes)
    })

In [35]:
lookalike_df = pd.DataFrame(data_for_csv, columns=['CustomerID', 'Lookalikes'])
lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike.csv has been created with top 3 lookalikes for each of the first 20 customers.")
print("\n=== SAMPLE OUTPUT ===")
print(lookalike_df.head(5))

Lookalike.csv has been created with top 3 lookalikes for each of the first 20 customers.

=== SAMPLE OUTPUT ===
  CustomerID                                         Lookalikes
0      C0001  [('C0120', 0.9383), ('C0192', 0.9031), ('C0181...
1      C0002  [('C0159', 0.9376), ('C0178', 0.9308), ('C0106...
2      C0003  [('C0152', 0.8922), ('C0031', 0.8809), ('C0085...
3      C0004  [('C0113', 0.9197), ('C0104', 0.8905), ('C0165...
4      C0005  [('C0007', 0.9643), ('C0140', 0.8651), ('C0186...
